In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig  # 导入 BitsAndBytesConfig

# 清理 GPU 缓存
torch.cuda.empty_cache()

# 配置4-bit量化参数
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, # 这是指模型的权重存储使用4位精度，可以大大减少模型占用的显存
    bnb_4bit_compute_dtype=torch.float16, # 这是指模型在计算时使用16位精度。计算：训练和推理
    bnb_4bit_use_double_quant=True, # 这是指在量化过程中使用双量化，可以进一步减少量化误差
    bnb_4bit_quant_type="nf4"  # 使用 normal float 4 量化类型。量化：将高精度（如32位）的浮点数转换为低精度（如4位）的浮点数，以减少模型占用的显存和提高计算速度。
)

# 指定自定义下载路径
cache_dir = "/root/autodl-tmp/gemma"

tokenizer = AutoTokenizer.from_pretrained(
    "google/gemma-2-9b",
    cache_dir=cache_dir
)

# 使用新的量化配置方式加载模型
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-9b",
    cache_dir=cache_dir,
    device_map="auto", # 自动管理模型在可用设备上的分配
    torch_dtype=torch.float16, # 使用float16精度
    quantization_config=quantization_config  # 使用量化配置
)

# Use a pipeline as a high-level helper
# from transformers import pipeline

# pipe = pipeline(
#     "text-generation",
#     model=model,  # 使用已加载的模型
#     tokenizer=tokenizer,  # 使用已加载的tokenizer
#     device_map="auto"
# )

# 确定设备（CPU或GPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# test
# tokenizer("你好"), tokenizer("你好", return_tensors="pt"), tokenizer.eos_token_id, tokenizer.decode(tokenizer.eos_token_id)
print(tokenizer("<start_of_turn>"), tokenizer("user"))
print(tokenizer("<end_of_turn>"), tokenizer("<eos>"))
print(tokenizer.decode(torch.tensor([2, 106])))
print(tokenizer.decode(torch.tensor([2, 1645])))


In [ ]:
def chat_with_gemma(prompt, max_new_tokens=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(device) # 返回的inputs是字典，包含input_ids, attention_mask。return_tensors="pt"返回的是tensor
    
    # 关于参数的设置请参考：https://huggingface.co/learn/cookbook/en/prompt_tuning_peft#inference-with-the-pre-trained-bloom-model
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature = 0.2,
        top_p=0.95,
        do_sample=True,
        repetition_penalty=1.5, # 对重复内容施加惩罚
        early_stopping=True, # The model can stop before reach the max_length
        eos_token_id=tokenizer.eos_token_id,
    )

    # 解码并处理回答
    response = tokenizer.decode(outputs[0][inputs['input_ids'].size(1):], skip_special_tokens=True) # inputs['input_ids'].size(1)是输入序列的长度
    return response

# 测试对话
USER_CHAT_TEMPLATE = "<start_of_turn>user\n{prompt}\n<end_of_turn><eos>\n"
MODEL_CHAT_TEMPLATE = "<start_of_turn>model\n{prompt}\n<end_of_turn><eos>\n"

user_prompt= "你好，请介绍一下自己。"
prompt = (
    USER_CHAT_TEMPLATE.format(prompt="你是谁？")
    + MODEL_CHAT_TEMPLATE.format(prompt="我是Gemma，一个由Google开发的智能助手。")
    # 开启新的对话
    + USER_CHAT_TEMPLATE.format(prompt=user_prompt)
    + "<start_of_turn>model\n" # 等待模型生成回答
)
print(prompt)
response = chat_with_gemma(prompt)
print("模型回答:\n", response)

# 简化prompt构造
# user_prompt = "你好，请介绍一下自己。"
# prompt = f"""<start_of_turn>user\n{user_prompt}<end_of_turn><eos>\n
#              <start_of_turn>model\n"""

# response = chat_with_gemma(prompt)
# print("模型回答:\n", response)

# Final

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig  # 导入 BitsAndBytesConfig

# 清理 GPU 缓存
torch.cuda.empty_cache()

# 配置4-bit量化参数
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, # 这是指模型的权重存储使用4位精度，可以大大减少模型占用的显存
    bnb_4bit_compute_dtype=torch.float16, # 这是指模型在计算时使用16位精度。计算：训练和推理
    bnb_4bit_use_double_quant=True, # 这是指在量化过程中使用双量化，可以进一步减少量化误差
    bnb_4bit_quant_type="nf4"  # 使用 normal float 4 量化类型。量化：将高精度（如32位）的浮点数转换为低精度（如4位）的浮点数，以减少模型占用的显存和提高计算速度。
)

# 指定自定义下载路径
cache_dir = "/root/autodl-tmp/gemma"

tokenizer = AutoTokenizer.from_pretrained(
    "google/gemma-2-9b",
    cache_dir=cache_dir
)

# 使用新的量化配置方式加载模型
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-9b",
    cache_dir=cache_dir,
    device_map="auto", # 自动管理模型在可用设备上的分配
    torch_dtype=torch.float16, # 使用float16精度
    quantization_config=quantization_config  # 使用量化配置
)

# 确定设备（CPU或GPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
def chat_with_gemma(prompt, max_new_tokens=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(device) # 返回的inputs是字典，包含input_ids, attention_mask。return_tensors="pt"返回的是tensor
    
    # 关于参数的设置请参考：https://huggingface.co/learn/cookbook/en/prompt_tuning_peft#inference-with-the-pre-trained-bloom-model
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature = 0.2,
        top_p=0.95,
        do_sample=True,
        repetition_penalty=1.5, # 对重复内容施加惩罚
        early_stopping=True, # The model can stop before reach the max_length
        eos_token_id=tokenizer.eos_token_id,
    )

    # 解码并处理回答
    response = tokenizer.decode(outputs[0][inputs['input_ids'].size(1):], skip_special_tokens=True) # inputs['input_ids'].size(1)是输入序列的长度
    return response

# 测试对话
USER_CHAT_TEMPLATE = "<start_of_turn>user\n{prompt}<end_of_turn><eos>\n"
MODEL_CHAT_TEMPLATE = "<start_of_turn>model\n{prompt}\n<end_of_turn><eos>\n"
user_prompt = "你好，请介绍一下自己。"
prompt = USER_CHAT_TEMPLATE.format(prompt=user_prompt)

# 单论对话
response = chat_with_gemma(prompt)
print(response)

# 多轮对话
# while True:
#     user_prompt = input("你: ")
#     prompt = USER_CHAT_TEMPLATE.format(prompt=user_prompt)
#     response = chat_with_gemma(prompt)
#     print("模型:", response)